# Kubectl

The `kubectl` utility is a primary way to interact with k8s cluster.

Different k8s resources have slightly different configurations. However, `kubectl` is designed in such a way that different resources often have many features in common. This page addresses the questions about managing the entire cluster with `kubectl`, as well as common details about managing different resource types. It is not always very detailed regarding questions specific to particular resources - there are specific pages for such details. 

## Install&Config

Check [installation guide](https://kubernetes.io/docs/tasks/tools/install-kubectl-linux/) for `kubectl`. For me, using the ubuntu option `snap install kubectl --classic` worked well.

To enable `TAB` completion add `source` the script printed with `kubectl completion bash`: `source <(kubectl completion bash)`. To enable completion at the start of the shell, add line into `~/.bashrc` file: `echo 'source <(kubectl completion bash)' >> ~/.bashrc`.

### Config file 

The behavior of the `kubectl` is typically defined by the configuration file that is located in the `~/.kube/config` directory.

---

The following cell displays a few lines of the `kubectl` configuration.

In [ ]:
cat ~/.kube/config | head -n 10

apiVersion: v1
clusters:
- cluster:
    certificate-authority: /home/user/.minikube/ca.crt
    extensions:
    - extension:
        last-update: Mon, 13 Oct 2025 09:07:40 CEST
        provider: minikube.sigs.k8s.io
        version: v1.37.0
      name: cluster_info


### Context

A k8s [context](kubernetes.io/docs/concepts/configuration/organize-cluster-access-kubeconfig/#context) is a configuration setup under a convenient name. You can switch between different `kubectl` configurations, which provide access to different clusters or different setups for the same cluster.

The following table shows useful `kubectl` commands that can be beneficial for managing the context.

| Command | Purpose |
| :--- | :--- |
| `kubectl config current-context` | Displays the name of the context that is currently active. |
| `kubectl config get-contexts` | Lists all the contexts defined in your kubeconfig file. |
| `kubectl config use-context <name>` | Sets the specified context as the current-context, switching the cluster/user/namespace your commands target. |
| `kubectl config set-context <name> --cluster=<cluster> --user=<user> --namespace=<namespace>` | Creates a new context or modifies an existing one to link a specific cluster, user, and/or default namespace. |
| `kubectl config delete-context <name>` | Deletes the specified context from your kubeconfig file. |

---

Minikube, which is primarily used as an example k8s setup, automatically creates a `kubectl` context for each profile. For example, consider how to select a `kubectl` context from the options provided by `minikube`.

In [2]:
minikube start --profile one-node &> /dev/null
minikube start --profile two-nodes &> /dev/null

The following cell shows the available `kubectl` contexts.

In [3]:
kubectl config get-contexts

CURRENT   NAME        CLUSTER     AUTHINFO    NAMESPACE
          one-node    one-node    one-node    default
*         two-nodes   two-nodes   two-nodes   default


The `two-nodes` context is marked as CURRENT. The following cell switches contexts and displays list of contexts list again.

In [4]:
kubectl config use-context one-node
kubectl config get-contexts

Switched to context "one-node".
CURRENT   NAME        CLUSTER     AUTHINFO    NAMESPACE
*         one-node    one-node    one-node    default
          two-nodes   two-nodes   two-nodes   default


Now there is different context specified as CURRENT.

In [5]:
minikube delete --profile one-node &> /dev/null
minikube delete --profile two-nodes &> /dev/null

## Cluster information

The `kubectl cluster-info` command displays the information about the cluster that `kubectl` currently connected to.

In [ ]:
kubectl cluster-info

Kubernetes control plane is running at https://192.168.49.2:8443
CoreDNS is running at https://192.168.49.2:8443/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


## Proxy

Since `kubectl` is just convenient wrapper around the Kubernetes API, it can communicate with the kubernetes API and allow you to create a locally hosted proxy for it. Use the `kubectl proxy` command for this. 

---

The following cell runs the proxy.

In [ ]:
kubectl proxy &
export proxy_pid=$!

[1] 101050


Now you are able to access api:

In [ ]:
curl localhost:8001/api

Starting to serve on 127.0.0.1:8001
{
  "kind": "APIVersions",
  "versions": [
    "v1"
  ],
  "serverAddressByClientCIDRs": [
    {
      "clientCIDR": "0.0.0.0/0",
      "serverAddress": "192.168.49.2:8443"
    }
  ]
}


In [ ]:
kill $proxy_pid

## Display resouces

The `kubectl get` command displays given resources. It have subcommands for all resources types. The most common are:

| Command                    | Resource Kind   | Description                                                              |
| -------------------------- | --------------- | ------------------------------------------------------------------------ |
| `kubectl get pods`         | **Pod**         | The smallest deployable unit — runs one or more containers.              |
| `kubectl get deployments`  | **Deployment**  | Manages Pods and ReplicaSets; handles rolling updates.                   |
| `kubectl get services`     | **Service**     | Exposes a set of Pods on a network endpoint (ClusterIP, NodePort, etc.). |
| `kubectl get replicasets`  | **ReplicaSet**  | Ensures a specified number of Pods are running.                          |
| `kubectl get daemonsets`   | **DaemonSet**   | Runs one Pod on each (or selected) node.                                 |
| `kubectl get statefulsets` | **StatefulSet** | Manages Pods with stable identities (useful for databases).              |
| `kubectl get jobs`         | **Job**         | Runs Pods to completion (batch jobs).                                    |
| `kubectl get cronjobs`     | **CronJob**     | Schedules Jobs to run periodically.                                      |
| `kubectl get nodes`        | **Node**        | Shows the cluster’s worker nodes.                                        |
| `kubectl get namespaces`   | **Namespace**   | Lists logical partitions of the cluster.                                 |

### Output format

The important thing is how you specify the output format. Use the `-o` option followed by the instructions on how to display the output.

There following types of instructions:

| Format             | Description                                           | Example                                                                       |
| ------------------ | ----------------------------------------------------- | ----------------------------------------------------------------------------- |
| `wide`             | Adds more columns (e.g., node name, IP).              | `kubectl get pods -o wide`                                                    |
| `name`             | Prints only resource names.                           | `kubectl get pods -o name` → `pod/nginx-1234`                                 |
| `yaml`             | Full YAML definition (like what’s stored in the API). | `kubectl get pod nginx -o yaml`                                               |
| `json`             | Same as above but in JSON.                            | `kubectl get pod nginx -o json`                                               |
| `jsonpath`         | Extracts specific fields using JSONPath syntax.       | `kubectl get pods -o jsonpath='{.items[*].metadata.name}'`                    |
| `jsonpath-as-json` | Like `jsonpath`, but ensures valid JSON output.       | `kubectl get pods -o jsonpath-as-json='{.items[*].metadata.name}'`            |
| `go-template`      | Uses Go templates to define custom output.            | `kubectl get pods -o go-template='{{.items[0].metadata.name}}'`               |
| `custom-columns`   | Displays only selected fields as table columns.       | `kubectl get pods -o custom-columns=NAME:.metadata.name,STATUS:.status.phase` |


---

For example, consider the output of the raw `kubectl get pods` command and the configuration option that shows additional information.

In [5]:
kubectl create deployment temp --image=nginx:1.22.1 --replicas=3 &> /dev/null

The following cell represents regular output:

In [7]:
kubectl get pods

NAME                    READY   STATUS    RESTARTS   AGE
temp-5dcd5945c4-255xn   1/1     Running   0          17s
temp-5dcd5945c4-rfvt7   1/1     Running   0          17s
temp-5dcd5945c4-rrm9p   1/1     Running   0          17s


The wide output adds some additional columns to the output table.

In [8]:
kubectl get pods -o wide

NAME                    READY   STATUS    RESTARTS   AGE   IP           NODE         NOMINATED NODE   READINESS GATES
temp-5dcd5945c4-255xn   1/1     Running   0          34s   10.244.0.4   daemon       <none>           <none>
temp-5dcd5945c4-rfvt7   1/1     Running   0          34s   10.244.1.3   daemon-m02   <none>           <none>
temp-5dcd5945c4-rrm9p   1/1     Running   0          34s   10.244.2.3   daemon-m03   <none>           <none>


In [9]:
kubectl delete deployment temp &> /dev/null

## Management types

The k8s supports two management types:

- The **imperative**: operates directly on live objects in a cluster.
- The **declarative**: operates through configuration files that declare the objects.

Check more in the [k8s Object Management](https://kubernetes.io/docs/concepts/overview/working-with-objects/object-management/) page of the official documentation.